In [1]:
import datetime as dt
import numpy as np
import pandas as pd
import os
from multiprocessing import  Pool
import gc
import matplotlib.pyplot as plt

In [2]:
import Indicator as ind

In [3]:
df1=pd.read_csv('./EU0520/EURUSD_ASK.csv')

In [10]:
def SMA(df,column,period):
    df[column+'Sma'+str(period)]=(df[column]).rolling(window=period).mean()
    return df

In [5]:
ind.SMA(df1,'Open',20)

Gmt time     Open     High      Low    Close  \
0      03.01.2005 00:00:00.000  1.35788  1.35797  1.35401  1.35454   
1      03.01.2005 01:00:00.000  1.35453  1.35491  1.35281  1.35364   
2      03.01.2005 02:00:00.000  1.35377  1.35377  1.34975  1.35028   
3      03.01.2005 03:00:00.000  1.35012  1.35014  1.33871  1.34081   
4      03.01.2005 04:00:00.000  1.34104  1.34436  1.34076  1.34422   
...                        ...      ...      ...      ...      ...   
99593  27.11.2020 17:00:00.000  1.19540  1.19572  1.19507  1.19569   
99594  27.11.2020 18:00:00.000  1.19569  1.19574  1.19503  1.19562   
99595  27.11.2020 19:00:00.000  1.19560  1.19631  1.19555  1.19579   
99596  27.11.2020 20:00:00.000  1.19580  1.19618  1.19565  1.19609   
99597  27.11.2020 21:00:00.000  1.19609  1.19650  1.19591  1.19650   

           Volume  OpenMacd12_26  OpenMacdSignal9  OpenSma20  
0      23339.4004       0.000000         0.000000        NaN  
1      27369.4004      -0.026724        -0.005345        NaN  
2      26765.0996      -0.053419        -0.014960        NaN  
3      23991.3008      -0.102842        -0.032536        NaN  
4      25356.6992      -0.212825        -0.068594        NaN  
...           ...            ...              ...        ...  
99593  11260.2100       0.081443         0.054145   1.192562  
99594   3368.9800       0.093668         0.062050   1.192772  
99595   4110.3501       0.101460         0.069932   1.193001  
99596   7462.6348       0.108005         0.077546   1.193235  
99597   6415.3550       0.114215         0.084880   1.193469  

[99598 rows x 9 columns]

In [12]:
SMA(df1,'Open',20)

Gmt time     Open     High      Low    Close  \
0      03.01.2005 00:00:00.000  1.35788  1.35797  1.35401  1.35454   
1      03.01.2005 01:00:00.000  1.35453  1.35491  1.35281  1.35364   
2      03.01.2005 02:00:00.000  1.35377  1.35377  1.34975  1.35028   
3      03.01.2005 03:00:00.000  1.35012  1.35014  1.33871  1.34081   
4      03.01.2005 04:00:00.000  1.34104  1.34436  1.34076  1.34422   
...                        ...      ...      ...      ...      ...   
99593  27.11.2020 17:00:00.000  1.19540  1.19572  1.19507  1.19569   
99594  27.11.2020 18:00:00.000  1.19569  1.19574  1.19503  1.19562   
99595  27.11.2020 19:00:00.000  1.19560  1.19631  1.19555  1.19579   
99596  27.11.2020 20:00:00.000  1.19580  1.19618  1.19565  1.19609   
99597  27.11.2020 21:00:00.000  1.19609  1.19650  1.19591  1.19650   

           Volume     Sma20  OpenSma20  
0      23339.4004       NaN        NaN  
1      27369.4004       NaN        NaN  
2      26765.0996       NaN        NaN  
3      23991.3008       NaN        NaN  
4      25356.6992       NaN        NaN  
...           ...       ...        ...  
99593  11260.2100  1.192562   1.192562  
99594   3368.9800  1.192772   1.192772  
99595   4110.3501  1.193001   1.193001  
99596   7462.6348  1.193235   1.193235  
99597   6415.3550  1.193469   1.193469  

[99598 rows x 8 columns]

In [4]:
df2=pd.read_csv('./EU0520/EURUSD_BID.csv')

In [5]:
df=pd.concat([df1,df2],axis=1)
df.columns=['Date','AskOpen','AskHigh' ,'AskLow','AskClose','AskVolume','Gmt time','BidOpen', 'BidHigh' ,'BidLow', 'BidClose' ,'BidVolume']

In [6]:
def PARALLELIZE(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [7]:
def MEAN (df):
    ##
    df['AskMean']=(df.AskClose+df.AskOpen+df.AskHigh+df.AskLow)/4.0
    ##
    df['BidMean']=(df.BidClose+df.BidOpen+df.BidHigh+df.BidLow)/4.0
    ##
    return df

In [8]:
def SMA(df):
    ##
    df['AskSma20']=df.AskClose.rolling(window=20).mean()
    df['AskSma50']=df.AskClose.rolling(window=50).mean()
    df['AskSma120']=df.AskClose.rolling(window=120).mean()
    ##
    df['BidSma20']=df.BidClose.rolling(window=20).mean()
    df['BidSma50']=df.BidClose.rolling(window=50).mean()
    df['BidSma120']=df.BidClose.rolling(window=120).mean()
    ##
    return df

In [7]:
def MACD (df):
    ##
    exp1 = df.AskClose.ewm(span=12, adjust=False).mean()
    exp2 = df.AskClose.ewm(span=26, adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=9, adjust=False).mean()
    df['AskMacd']=macd*100
    df['AskMacdSignal']=exp3*100
    ##
    exp1 = df.BidClose.ewm(span=12, adjust=False).mean()
    exp2 = df.BidClose.ewm(span=26, adjust=False).mean()
    macd = exp1-exp2
    exp3 = macd.ewm(span=9, adjust=False).mean()
    df['BidMacd']=macd*100
    df['BidMacdSignal']=exp3*100
    ##
    return df

In [10]:
def MINMAX(df):
    ##
    df['AskMin120']=df.AskLow.rolling(window=120).min()
    df['AskMax120']=df.AskHigh.rolling(window=120).max()
    ##
    df['BidMin120']=df.BidLow.rolling(window=120).min()
    df['BidMax120']=df.BidHigh.rolling(window=120).max()
    ##
    return df

In [11]:
def STANDERDDEVIATION(df):
    ##
    df['AskStd20Scaled100']=df.AskClose.rolling(window=20).std()*100
    ##
    df['BidStd20Scaled100']=df.BidClose.rolling(window=20).std()*100
    ##
    return df

In [12]:
def CCI(df):
    ##
    df['AskCci']=(df.AskMean - df.AskMean.rolling(window=20).mean()) / (1.5 * df.AskMean.rolling(window=20).std())
    ##
    df['BidCci']=(df.BidMean - df.BidMean.rolling(window=20).mean()) / (1.5 * df.BidMean.rolling(window=20).std())
    ##
    return df

In [17]:
def RSI (df):
    ##
    df['AskGain']=(df.AskClose-df.AskClose.shift(periods=1)).clip(lower=0)
    df['AskLoss']=(df.AskClose.shift(periods=1)-df.AskClose).clip(lower=0)
    RSAsk=df.AskGain.rolling(window=14).mean()/df.AskLoss.rolling(window=14).mean()
    df['AskRsi']=100-100/(1+RSAsk)
    ##
    df['BidGain']=(df.BidClose-df.BidClose.shift(periods=1)).clip(lower=0)
    df['BidLoss']=(df.BidClose.shift(periods=1)-df.BidClose).clip(lower=0)
    RSBid=df.BidGain.rolling(window=14).mean()/df.BidLoss.rolling(window=14).mean()
    df['AskRsi']=100-100/(1+RSBid)
    ##
    return df

In [18]:
def STOCASTIC(df):
    ##
    df['Ask%K']=(df.AskClose-df.AskLow.rolling(window=14).min())/(df.AskHigh.rolling(window=14).max()-df.AskLow.rolling(window=14).min())
    df['Ask%D']=(df['Ask%K']).rolling(window=3).mean()
    ##
    df['Bid%K']=(df.BidClose-df.BidLow.rolling(window=14).min())/(df.BidHigh.rolling(window=14).max()-df.BidLow.rolling(window=14).min())
    df['Bid%D']=(df['Bid%K']).rolling(window=3).mean()
    ##
    return df

In [21]:
def PARABOLICFIT(A,i,n):
    X=np.arange(len(A))
    B=np.polyfit(X,A,n)
    return B[i]    

In [22]:
def FITTING(df):
    ##
    df['AskSlope']=df.AskClose.rolling(window=14).apply(PARABOLICFIT,raw=True,args=(1,1))

In [13]:
def MODIFY(df):
    df.drop(columns='Gmt time',inplace=True)
    df['Date'] = pd.to_datetime(df['Date'],format='%d.%m.%Y %H:%M:%S.%f') 
    df['RawSpread']=df.AskClose-df.BidClose
    df['Spread']=2/(1+np.exp(df.RawSpread*1000))
    df.AskVolume=2*df.AskVolume/10**6
    df.BidVolume=2*df.BidVolume/10**6
    return df

In [14]:
df=PARALLELIZE(df,PREDICT)

In [15]:
df=PARALLELIZE(df,MEAN)

In [16]:
df=PARALLELIZE(df,SMA)

In [8]:
df=PARALLELIZE(df,MACD)

In [18]:
df=PARALLELIZE(df,MINMAX)

In [19]:
df=PARALLELIZE(df,STANDERDDEVIATION)

In [20]:
df=PARALLELIZE(df,CCI)

In [21]:
df=PARALLELIZE(df,MODIFY)

In [22]:
df=df.dropna()

In [23]:
df.reset_index(inplace=True,drop=True)

In [24]:
df.to_csv('Dataset.csv')

In [9]:
df

Date  AskOpen  AskHigh   AskLow  AskClose  \
0      03.01.2005 00:00:00.000  1.35788  1.35797  1.35401   1.35454   
1      03.01.2005 01:00:00.000  1.35453  1.35491  1.35281   1.35364   
2      03.01.2005 02:00:00.000  1.35377  1.35377  1.34975   1.35028   
3      03.01.2005 03:00:00.000  1.35012  1.35014  1.33871   1.34081   
4      03.01.2005 04:00:00.000  1.34104  1.34436  1.34076   1.34422   
...                        ...      ...      ...      ...       ...   
99593  27.11.2020 17:00:00.000  1.19540  1.19572  1.19507   1.19569   
99594  27.11.2020 18:00:00.000  1.19569  1.19574  1.19503   1.19562   
99595  27.11.2020 19:00:00.000  1.19560  1.19631  1.19555   1.19579   
99596  27.11.2020 20:00:00.000  1.19580  1.19618  1.19565   1.19609   
99597  27.11.2020 21:00:00.000  1.19609  1.19650  1.19591   1.19650   

        AskVolume                 Gmt time  BidOpen  BidHigh   BidLow  \
0      23339.4004  03.01.2005 00:00:00.000  1.35772  1.35777  1.35391   
1      27369.4004  03.01.2005 01:00:00.000  1.35443  1.35477  1.35271   
2      26765.0996  03.01.2005 02:00:00.000  1.35357  1.35358  1.34965   
3      23991.3008  03.01.2005 03:00:00.000  1.34998  1.35004  1.33856   
4      25356.6992  03.01.2005 04:00:00.000  1.34094  1.34422  1.34056   
...           ...                      ...      ...      ...      ...   
99593  11260.2100  27.11.2020 17:00:00.000  1.19537  1.19569  1.19504   
99594   3368.9800  27.11.2020 18:00:00.000  1.19566  1.19570  1.19498   
99595   4110.3501  27.11.2020 19:00:00.000  1.19558  1.19628  1.19551   
99596   7462.6348  27.11.2020 20:00:00.000  1.19574  1.19615  1.19559   
99597   6415.3550  27.11.2020 21:00:00.000  1.19605  1.19641  1.19589   

       BidClose   BidVolume   AskMacd  AskMacdSignal   BidMacd  BidMacdSignal  
0       1.35438  29752.0996  0.000000       0.000000  0.000000       0.000000  
1       1.35354  33412.1992 -0.007179      -0.001436 -0.006701      -0.001340  
2       1.35012  31876.0996 -0.039526      -0.009054 -0.039157      -0.008903  
3       1.34061  29160.5996 -0.139963      -0.035236 -0.140002      -0.035123  
4       1.34418  30099.0996 -0.189855      -0.066159 -0.188937      -0.065886  
...         ...         ...       ...            ...       ...            ...  
99593   1.19565  13426.1797  0.093820       0.062170  0.093947       0.062373  
99594   1.19559   5119.9702  0.101773       0.070091  0.101935       0.070285  
99595   1.19574  10270.4805  0.108200       0.077712  0.108227       0.077874  
99596   1.19604   6402.1499  0.114395       0.085049  0.114318       0.085163  
99597   1.19620   4916.5190  0.121216       0.092282  0.119063       0.091943  

[99598 rows x 16 columns]